In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from datetime import datetime

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

pio.renderers.default = "notebook"
pio.templates.default = "plotly_white"


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

In [3]:
df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='both')

In [5]:
def show_response_contents(df):
    print("The response contains:")
    print(json.dumps(list(df['variable'].unique()), sort_keys=True, indent=4))
    print(json.dumps(list(df['device'].unique()), sort_keys=True, indent=4))

In [7]:
df = df.sort_values(by=['variable','datetime'])
df = pro.datetime_attributes(df)

df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='left')
df_bl['Periodo'] = 'Baseline'
df_st['Periodo'] = 'Estudio'

past_w = df_bl.loc[cfg.PAST_WEEK[0]:cfg.PAST_WEEK[1]]

# df_cons = df.query("variable == 'front-consumo-activa'")
# df_ea = cln.recover_energy_from_consumption(df_cons, new_varname='front-energia-activa-acumulada')
# df_pa_synth = cln.differentiate_single_variable(df_ea, 'front-potencia-activa-sintetica', remove_gap_data=True)
# df_ea_interp = cln.linearly_interpolate_series(df_ea, data_rate_in_minutes=None)

In [8]:
df_bl

,value,variable,device,device_name,hour,day,dow,cont_dow,week,month,year,Periodo
datetime,,,,,,,,,,,,
2023-01-08 00:00:00-05:00,0.00,aa-consumo-activa,bc-sede-twins-bay,BC - Sede Adm Twins bay,0,8,domingo,6.000000,1,1,2023,Baseline
2023-01-08 00:00:00-05:00,0.38,aa-consumo-activa,bc---s.a.-valledupar,BC - S.A. Valledupar,0,8,domingo,6.000000,1,1,2023,Baseline
2023-01-08 00:00:00-05:00,9.83,aa-consumo-activa,bc---s.a.-granada,BC - S.A. Granada,0,8,domingo,6.000000,1,1,2023,Baseline
2023-01-08 00:00:00-05:00,1.99,aa-consumo-activa,bc-816-1---calle-10,BC 816 - Calle 10,0,8,domingo,6.000000,1,1,2023,Baseline
2023-01-08 00:00:00-05:00,0.09,aa-consumo-activa,bc71-2,BC 793 - Profesionales,0,8,domingo,6.000000,1,1,2023,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-07 23:57:00-05:00,15.50,tr,bc30,BC 792 - Paseo del comercio,23,7,domingo,6.958333,18,5,2023,Baseline
2023-05-07 23:57:00-05:00,37.00,tr,bc26,BC 824 - Ventura Plaza,23,7,domingo,6.958333,18,5,2023,Baseline
2023-05-07 23:57:00-05:00,17.00,tr,bc21,BC 459 - Campo Alegre,23,7,domingo,6.958333,18,5,2023,Baseline


In [42]:
df_bl_lunes = df_bl[df_bl['dow'] == 'lunes']

In [43]:
festivos = ['2023-01-09', '2023-03-20', '2023-05-01']
df_bl_lunes_festivos = df_bl_lunes[df_bl_lunes.index.strftime('%Y-%m-%d').isin(festivos)]

In [44]:
consumo_promedio_lunes = df_bl_lunes.groupby('device_name')['value'].mean()

In [45]:
consumo_promedio_lunes_festivos = df_bl_lunes_festivos.groupby('device_name')['value'].mean()

In [46]:
# Paso 1: Filtrar los datos para incluir solo los días lunes
df_bl_lunes = df_bl[df_bl['dow'] == 'lunes']

# Paso 2: Filtrar los datos para incluir solo los días lunes festivos
festivos = ['2023-01-09', '2023-03-20', '2023-05-01']
df_bl_lunes_festivos = df_bl_lunes[df_bl_lunes.index.strftime('%Y-%m-%d').isin(festivos)]

# Paso 3: Calcular el consumo promedio de energía para los días lunes no festivos, agrupado por "device_name"
consumo_promedio_lunes = df_bl_lunes.groupby('device_name')['value'].mean()

# Paso 4: Calcular el consumo promedio de energía para los días lunes festivos, agrupado por "device_name"
consumo_promedio_lunes_festivos = df_bl_lunes_festivos.groupby('device_name')['value'].mean()

# Imprimir los resultados
print("Consumo promedio de los días lunes (no festivos):")
print(consumo_promedio_lunes)
print()
print("Consumo promedio de los días lunes festivos:")
print(consumo_promedio_lunes_festivos)


Consumo promedio de los días lunes (no festivos):
device_name
BC - S.A. Granada                      1008.934594
BC - S.A. Valledupar                    276.864672
BC - Sede Adm Twins bay                  36.344806
BC 044 - Piedecuesta                     14.498711
BC 061 - Carrera Primera                 95.388037
BC 185 - Llano Grande Palmira            10.864937
BC 197 - Guatapuri                        8.010136
BC 205 - Villa Colombia                  10.604225
BC 210 - Banca Colombia Cartagena        15.356667
BC 216 - Sabana de Torres                11.542434
BC 253 - Puerta del Norte                 7.056277
BC 291 - Las Palmas                      16.219320
BC 302 - Quebrada Seca                   12.255266
BC 306 - Barrancabermeja                 94.899583
BC 311 - Bello                           74.274850
BC 371 - Caucasia                        10.676875
BC 398 - Mariquita                        7.776508
BC 424 - Honda                           10.155634
BC 453 - Pitalito   

In [41]:
# Paso 1: Calcular la variación porcentual entre los días lunes festivos y los días lunes normales para cada sede
variacion_porcentual = ((consumo_promedio_lunes_festivos - consumo_promedio_lunes) / consumo_promedio_lunes) * 100

# Paso 2: Ordenar los resultados de manera descendente
variacion_porcentual_ordenada = variacion_porcentual.sort_values(ascending=False)

# Paso 3: Imprimir los resultados
print("Variación porcentual del consumo en los días lunes festivos respecto a los días lunes normales:")
print(variacion_porcentual_ordenada)


Variación porcentual del consumo en los días lunes festivos respecto a los días lunes normales:
device_name
BC - S.A. Granada                     NaN
BC - S.A. Valledupar                  NaN
BC - Sede Adm Twins bay               NaN
BC 044 - Piedecuesta                  NaN
BC 061 - Carrera Primera              NaN
BC 185 - Llano Grande Palmira         NaN
BC 197 - Guatapuri                    NaN
BC 205 - Villa Colombia               NaN
BC 210 - Banca Colombia Cartagena     NaN
BC 216 - Sabana de Torres             NaN
BC 253 - Puerta del Norte             NaN
BC 291 - Las Palmas                   NaN
BC 302 - Quebrada Seca                NaN
BC 306 - Barrancabermeja              NaN
BC 311 - Bello                        NaN
BC 371 - Caucasia                     NaN
BC 398 - Mariquita                    NaN
BC 424 - Honda                        NaN
BC 453 - Pitalito                     NaN
BC 459 - Campo Alegre                 NaN
BC 495 - El Bosque                    NaN
BC 496 - I

In [38]:
consumo_promedio_lunes = df_bl[df_bl["dow"] == "lunes"].groupby("device_name")["value"].mean()

In [39]:
dias_festivos = ['2023-01-09', '2023-03-20', '2023-05-01']
df_lunes_festivos = df_bl[df_bl.index.isin(dias_festivos) & (df_bl["dow"] == "lunes")]

In [40]:
consumo_promedio_lunes_festivos = df_lunes_festivos.groupby("device_name")["value"].mean()

In [36]:
df_resultado = pd.DataFrame(consumo_promedio_lunes, columns=['Consumo Lunes'])

In [25]:
print(df_resultado)


Empty DataFrame
Columns: [Consumo Lunes, Consumo Lunes Festivo, Diferencia, Diferencia Porcentual]
Index: []
